# Skin lesion classification of dermoscopic images using machine learning and convolutional neural network

19 December 2022

https://www.nature.com/articles/s41598-022-22644-9#Tab7

https://aihub.or.kr/aihubdata/data/view.do?currMenu=&topMenu=&aihubDataSe=realm&dataSetSn=561

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Preprocessing

In [14]:
import cv2
import glob
import json
import numpy as np
import os
from tqdm import tqdm
import re
import pandas as pd

In [19]:
def get_image_and_json_paths(src_path):
    image_paths = sorted(glob.glob(os.path.join(src_path, '**', '*.jpg'), recursive=True))
    json_paths = sorted(glob.glob(os.path.join(src_path, '**', '*.json'), recursive=True))

    return image_paths, json_paths


def get_image_data(image_paths):
    return [cv2.imread(image_path) for image_path in tqdm(image_paths, desc='Loading Images')]

def extract_metadata_and_locations_from_json(json_paths):
    meta_data, polygon_data, box_data = [], [], []
    for json_path in tqdm(json_paths, desc='Loading JSON', unit=' file'):
        with open(json_path, "r", encoding="utf-8") as file:
            json_data = json.loads(re.sub(r'[\x00-\x1F\x7F-\x9F]', '', file.read()))
            labeling_info, metadata = json_data['labelingInfo'], json_data.get('metaData', None)

            filtered_metadata = {
                'breed': metadata.get('breed', None),
                'age': metadata.get('age', None),
                'gender': metadata.get('gender', None),
                'region': metadata.get('region', None),
                'lesions': metadata.get('lesions', None)
            }

            for entry in labeling_info:
                if 'polygon' in entry:
                    polygon_data.append(entry['polygon'].get('location', None))
                if 'box' in entry:
                    box_data.append(entry['box'].get('location', None))

            meta_data.append(filtered_metadata)

    return meta_data, polygon_data, box_data

def create_mask_maps(image_data, polygon_data, box_data):
    mask_maps = []

    for image in tqdm(image_data, desc='Generating Mask Maps for Each Image'):
        mask_map = np.zeros(image.shape[:2] + (4,), dtype=np.uint8)
        fill_value = 1

        for entry in tqdm(polygon_data, desc='Drawing Polygon'):
            if 'polygon' in entry:
                loc = entry['polygon']['location']
                if loc:
                    loc_array = [[coord['x'], coord['y']] for coord in loc]
                    polygon_points = np.array(loc_array, np.int32).reshape((-1, 1, 2))
                    cv2.fillPoly(mask_map[..., 0], [polygon_points], 255)
                    cv2.fillPoly(mask_map[..., 1], [polygon_points], fill_value)

        for entry in tqdm(box_data, desc='Drawing Box'):
            if 'box' in entry:
                loc = entry['box']['location']
                if loc:
                    cv2.rectangle(mask_map[..., 2], tuple(loc[:2]), tuple(loc[2:]), 255, thickness=-1)
                    cv2.rectangle(mask_map[..., 3], tuple(loc[:2]), tuple(loc[2:]), fill_value, thickness=-1)

        mask_maps.append(mask_map)

    return mask_maps

def resize_images(images, width, height):
    return [cv2.resize(image, (width, height), interpolation=cv2.INTER_AREA) for image in tqdm(images, desc='Resizing Images')]

In [20]:
src_path = "/content/drive/Shareddrives/반려견"

In [21]:
image_paths, json_paths = get_image_and_json_paths(src_path)

In [22]:
image_data = get_image_data(image_paths)

Loading Images: 100%|██████████| 1425/1425 [00:48<00:00, 29.60it/s]


In [23]:
image_data[:5]

[array([[[  6,  34,  45],
         [  9,  37,  48],
         [ 13,  41,  52],
         ...,
         [ 65,  94,  91],
         [ 54,  83,  80],
         [ 49,  78,  75]],
 
        [[  5,  33,  44],
         [  8,  36,  47],
         [ 12,  40,  51],
         ...,
         [ 64,  93,  90],
         [ 68,  97,  94],
         [ 73, 102,  99]],
 
        [[  5,  33,  44],
         [  6,  34,  45],
         [  9,  37,  48],
         ...,
         [ 67,  94,  91],
         [ 79, 106, 103],
         [ 88, 115, 112]],
 
        ...,
 
        [[ 13,   9,  21],
         [  9,   5,  17],
         [  5,   1,  13],
         ...,
         [137, 116, 131],
         [138, 117, 132],
         [139, 118, 133]],
 
        [[ 18,  13,  28],
         [ 13,   8,  23],
         [  9,   4,  19],
         ...,
         [141, 120, 135],
         [142, 121, 136],
         [142, 121, 136]],
 
        [[ 31,  26,  41],
         [ 27,  22,  37],
         [ 22,  17,  32],
         ...,
         [139, 118, 133],
  

In [24]:
meta_data, polygon_data, box_data = extract_metadata_and_locations_from_json(json_paths)

Loading JSON: 100%|██████████| 1424/1424 [00:01<00:00, 1056.61 file/s]


In [25]:
meta_data[:5]

[{'breed': '푸들', 'age': '10', 'gender': 'M', 'region': 'B', 'lesions': 'A6'},
 {'breed': '몽그렐', 'age': '10', 'gender': 'M', 'region': 'L', 'lesions': 'A6'},
 {'breed': '치와와', 'age': '9', 'gender': 'F', 'region': 'B', 'lesions': 'A6'},
 {'breed': '몽그렐', 'age': '9', 'gender': 'M', 'region': 'B', 'lesions': 'A6'},
 {'breed': '비숑프리제', 'age': '8', 'gender': 'F', 'region': 'L', 'lesions': 'A6'}]

In [26]:
polygon_data[:5]

[[{'x1': 1609,
   'y1': 1061,
   'x2': 1655,
   'y2': 1064,
   'x3': 1691,
   'y3': 1063,
   'x4': 1748,
   'y4': 1039,
   'x5': 1754,
   'y5': 1025,
   'x6': 1727,
   'y6': 1007,
   'x7': 1646,
   'y7': 1003,
   'x8': 1576,
   'y8': 1000,
   'x9': 1525,
   'y9': 1015,
   'x10': 1513,
   'y10': 1025,
   'x11': 1513,
   'y11': 1043,
   'x12': 1520,
   'y12': 1046,
   'x13': 1609,
   'y13': 1061}],
 [{'x1': 950,
   'y1': 625,
   'x2': 988,
   'y2': 608,
   'x3': 1037,
   'y3': 581,
   'x4': 1064,
   'y4': 524,
   'x5': 1033,
   'y5': 458,
   'x6': 941,
   'y6': 430,
   'x7': 835,
   'y7': 440,
   'x8': 830,
   'y8': 496,
   'x9': 845,
   'y9': 574,
   'x10': 854,
   'y10': 595,
   'x11': 854,
   'y11': 595,
   'x12': 856,
   'y12': 596,
   'x13': 950,
   'y13': 625}],
 [{'x1': 467,
   'y1': 550,
   'x2': 541,
   'y2': 556,
   'x3': 577,
   'y3': 551,
   'x4': 617,
   'y4': 521,
   'x5': 625,
   'y5': 487,
   'x6': 605,
   'y6': 442,
   'x7': 548,
   'y7': 437,
   'x8': 521,
   'y8': 461,

In [27]:
box_data[:5]

[[{'x': 1513, 'y': 1000, 'width': 241, 'height': 64}],
 [{'x': 830, 'y': 430, 'width': 234, 'height': 195}],
 [{'x': 467, 'y': 437, 'width': 158, 'height': 119}],
 [{'x': 632, 'y': 386, 'width': 254, 'height': 171}],
 [{'x': 599, 'y': 398, 'width': 189, 'height': 212}]]

In [ ]:
masks_maps = create_mask_maps(image_data, polygon_data, box_data)

Streaming output truncated to the last 5000 lines.
Drawing Polygon: 100%|██████████| 1449/1449 [00:00<00:00, 1436093.22it/s]

Drawing Polygon: 100%|██████████| 1449/1449 [00:00<00:00, 966223.61it/s]

Drawing Box: 100%|██████████| 1449/1449 [00:00<00:00, 1528942.51it/s]

Drawing Polygon: 100%|██████████| 1449/1449 [00:00<00:00, 1002729.99it/s]

Drawing Box: 100%|██████████| 1449/1449 [00:00<00:00, 1590564.38it/s]

Drawing Polygon: 100%|██████████| 1449/1449 [00:00<00:00, 1128187.58it/s]

Drawing Box: 100%|██████████| 1449/1449 [00:00<00:00, 926596.51it/s]

Drawing Polygon: 100%|██████████| 1449/1449 [00:00<00:00, 1376879.59it/s]

Drawing Box: 100%|██████████| 1449/1449 [00:00<00:00, 1510324.68it/s]

Drawing Polygon: 100%|██████████| 1449/1449 [00:00<00:00, 1085082.40it/s]

Drawing Polygon: 100%|██████████| 1449/1449 [00:00<00:00, 1014954.32it/s]

Drawing Box: 100%|██████████| 1449/1449 [00:00<00:00, 1086246.02it/s]

Drawing Polygon: 100%|██████████| 1449/1449 [00:00<00:00, 1019893.69it/

In [18]:
first_channel_for_first_5_images = [mask_map[..., 0] for mask_map in masks_maps]

TypeError: ignored

In [ ]:
original_size_mask_maps = resize_images(masks_maps, 96, 96)

Resizing Images: 100%|██████████| 750/750 [00:08<00:00, 89.06it/s]


In [ ]:
original_size_mask_maps

[array([[[0, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 0],
         ...,
         [0, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 0]],
 
        [[0, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 0],
         ...,
         [0, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 0]],
 
        [[0, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 0],
         ...,
         [0, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 0]],
 
        ...,
 
        [[0, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 0],
         ...,
         [0, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 0]],
 
        [[0, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 0],
         ...,
         [0, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 0]],
 
        [[0, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 0],
         ...,
         [0, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 0]]], dtype=uint8),
 array([[[0, 0, 0, 0],
         [0, 0, 0, 

In [ ]:
resized_mask_maps = create_mask_maps(resize_images(image_data, 96, 96), polygon_data, box_data)

Drawing Polygon:   0%|          | 0/764 [00:00<?, ?it/s]
                                                        

Drawing Box:   0%|          | 0/764 [00:00<?, ?it/s]

                                                    
Drawing Polygon:   0%|          | 0/764 [00:00<?, ?it/s]
                                                        

Drawing Box:   0%|          | 0/764 [00:00<?, ?it/s]

                                                    
Drawing Polygon:   0%|          | 0/764 [00:00<?, ?it/s]
                                                        

Drawing Box:   0%|          | 0/764 [00:00<?, ?it/s]

                                                    
Drawing Polygon:   0%|          | 0/764 [00:00<?, ?it/s]
                                                        

Drawing Box:   0%|          | 0/764 [00:00<?, ?it/s]

                                                    
Drawing Polygon:   0%|          | 0/764 [00:00<?, ?it/s]
                                                       

In [ ]:
resized_mask_maps

[array([[[0, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 0],
         ...,
         [0, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 0]],
 
        [[0, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 0],
         ...,
         [0, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 0]],
 
        [[0, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 0],
         ...,
         [0, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 0]],
 
        ...,
 
        [[0, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 0],
         ...,
         [0, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 0]],
 
        [[0, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 0],
         ...,
         [0, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 0]],
 
        [[0, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 0],
         ...,
         [0, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 0]]], dtype=uint8),
 array([[[0, 0, 0, 0],
         [0, 0, 0, 

In [ ]:
metadata_df = pd.DataFrame(meta_data)

In [ ]:
metadata_df

,breed,age,gender,region,lesions,polygon_location,box_location
0,포메라니안,8,F,B,A7,[],[]
1,푸들,1,M,L,A7,[],[]
2,푸들,1,M,L,A7,[],[]
3,푸들,1,M,H,A7,[],[]
4,푸들,1,M,B,A7,[],[]
...,...,...,...,...,...,...,...
744,푸들,3,F,H,A3,[],[]
745,푸들,3,F,H,A3,[],[]
746,푸들,3,F,H,A3,[],[]
747,푸들,3,F,B,A3,[],[]


# Modeling

In [ ]:
import tensorflow as tf
from tensorflow.keras import models
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
# train, test set 분할
test_size = 0.2
temp_img_data, test_img_data, temp_meta_data, test_meta_data, temp_orig_size_maps, test_orig_size_maps, temp_resized_maps, test_resized_maps = train_test_split(
    image_data, metadata_df, original_size_mask_maps, resized_mask_maps,
    test_size=test_size, random_state=42
)

# test set을 기준으로 적용된 비율로 val_size 계산
val_size = 0.2
adjusted_val_size = val_size / (1.0 - test_size)

# train, validation set 분할
train_img_data, val_img_data, train_meta_data, val_meta_data, train_orig_size_maps, val_orig_size_maps, train_resized_maps, val_resized_maps = train_test_split(
    temp_img_data, temp_meta_data, temp_orig_size_maps, temp_resized_maps,
    test_size=adjusted_val_size, random_state=42
)

In [ ]:
# Sequential 모델 생성
model = models.Sequential()

# 첫번째 Conv2D 레이어
model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same', input_shape=(96,96,3)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(3, 3)))

# 첫번째 Dropout 레이어
model.add(Dropout(0.25))

# 두번째 Conv2D 레이어
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

# 두번째 Dropout 레이어
model.add(Dropout(0.25))

# 세번째 Conv2D 레이어
model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

# 세번째 Dropout 레이어
model.add(Dropout(0.25))

# Flatten 레이어
model.add(Flatten())

# 첫번째 Dense 레이어
model.add(Dense(units=1024, activation='relu'))
model.add(BatchNormalization())

# 두번째 Dropout 레이어
model.add(Dropout(0.5))

# 두번째 Dense 레이어: 최종 출력 레이어
model.add(Dense(units=7, activation='softmax'))

# 모델 컴파일
opt = Adam(lr=0.001, decay=0.00001)
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

# 모델 구조 요약
model.summary()

# 모델 학습
epochs = 150
batch_size = 32

history = model.fit(train_data, epochs=epochs, batch_size=batch_size, validation_data=val_data)

# 모델 평가
test_loss, test_acc = model.evaluate(test_data)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_acc)

In [ ]:
# TensorFlow Lite 모델로 변환
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# 변환된 모델을 파일로 저장
with open('your_model.tflite', 'wb') as f:
    f.write(tflite_model)